In [1]:
# Import
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(threshold=np.nan)
import tensorflow as tf
import os
import random
import sys
import utils
from sklearn.metrics import confusion_matrix

In [2]:
# Load old model
load_model = False

In [3]:
# Read training data
filename = "data_train.txt"

try:
    infile = open(filename, 'r')
except IOError as error:
    sys.stderr.write("File I/O error, reason" + str(error) + "\n")
    sys.exit(1)

seqflag = False
X_train = []
Y_train = []
seq_aa = []
seq_ss = []
for line in infile:
    if line.startswith("end") or line.startswith("<end>"):
        seqflag = False
        X_train.append(seq_aa)
        Y_train.append(seq_ss)
        seq_aa = []
        seq_ss = []
    elif seqflag:
        aa = line.split()[0]
        ss = line.split()[1]
        seq_aa.append(aa)
        seq_ss.append(ss)
    elif line.startswith("<>"):
        seqflag = True
        
# Read validation data
filename = "data_valid.txt"

try:
    infile = open(filename, 'r')
except IOError as error:
    sys.stderr.write("File I/O error, reason" + str(error) + "\n")
    sys.exit(1)

seqflag = False
X_valid = []
Y_valid = []
seq_aa = []
seq_ss = []
for line in infile:
    if line.startswith("end") or line.startswith("<end>"):
        seqflag = False
        X_valid.append(seq_aa)
        Y_valid.append(seq_ss)
        seq_aa = []
        seq_ss = []
    elif seqflag:
        aa = line.split()[0]
        ss = line.split()[1]
        seq_aa.append(aa)
        seq_ss.append(ss)
    elif line.startswith("<>"):
        seqflag = True

In [4]:
# Define encoding dictionaries
aadict = {'A':[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
          'R':[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
          'N':[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
          'D':[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
          'C':[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
          'Q':[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
          'E':[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
          'G':[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
          'H':[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
          'I':[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
          'L':[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
          'K':[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
          'M':[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
          'F':[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
          'P':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
          'S':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],                                    
          'T':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
          'W':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
          'Y':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
          'V':[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
 }

ssdict = {'_':[1,0,0], 
          'e':[0,1,0],
          'h':[0,0,1],
 }
           
# One-hot-encode training data
for i in range(0,len(X_train)):
    seq_aa = X_train[i]
    seq_ss = Y_train[i]

    for j in range(0,len(seq_ss)):
        aa = seq_aa[j]
        ss = seq_ss[j]
        
        if aa not in aadict:
            print("Unknown aa: " + aa)
            sys.exit(1)
        else:
            X_train[i][j] = aadict[aa]
        
        if ss not in ssdict:
            print("Unknown ss: " + ss)
            sys.exit(1)
        else:
            Y_train[i][j] = ssdict[ss]

# One-hot-encode validation data
for i in range(0,len(X_valid)):
    seq_aa = X_valid[i]
    seq_ss = Y_valid[i]

    for j in range(0,len(seq_ss)):
        aa = seq_aa[j]
        ss = seq_ss[j]
        
        if aa not in aadict:
            print("Unknown aa: " + aa)
            sys.exit(1)
        else:
            X_valid[i][j] = aadict[aa]
        
        if ss not in ssdict:
            print("Unknown ss: " + ss)
            sys.exit(1)
        else:
            Y_valid[i][j] = ssdict[ss]
            
# Do padding by adding half window size to each end
windowsize = 17
padding = int((windowsize-1)/2)

for i in range(len(X_train)):
    X_train[i] = np.vstack((np.zeros((padding,20)),X_train[i],np.zeros((padding,20))))

for i in range(len(X_valid)):
    X_valid[i] = np.vstack((np.zeros((padding,20)),X_valid[i],np.zeros((padding,20))))

# Collapse data structures to long array of  concatenated sequences
X_train = np.vstack(X_train)
Y_train = np.vstack(Y_train)

X_valid = np.vstack(X_valid)
Y_valid = np.vstack(Y_valid)

# Window slicing
X_train_windows = np.zeros([len(X_train)-windowsize+1,windowsize*20])
X_valid_windows = np.zeros([len(X_valid)-windowsize+1,windowsize*20])

for i in range(len(X_train_windows)):
    X_train_windows[i,:] = np.reshape(X_train[i:i+windowsize,:],[1,windowsize*20])

for i in range(len(X_valid_windows)):
    X_valid_windows[i,:] = np.reshape(X_valid[i:i+windowsize,:],[1,windowsize*20])
    
# Masking
boolmask_train = np.sum(X_train_windows[:,padding*20:padding*20+20],axis=1) > 0 #Check that central (predicted) aa is not zero
boolmask_valid = np.sum(X_valid_windows[:,padding*20:padding*20+20],axis=1) > 0 

X_train_windows_masked = X_train_windows[boolmask_train,:]
X_valid_windows_masked = X_valid_windows[boolmask_valid,:]

In [5]:
## Build the network
tf.reset_default_graph()

## Define placeholders
features = windowsize*20
num_classes = 3

X_ph = tf.placeholder(tf.float32, [None,features], name='xPlaceholder')
Y_ph = tf.placeholder(tf.float32, [None,num_classes], name='yPlaceholder')

## Define the model

# Initialize weights
weight_initializer = tf.truncated_normal_initializer(stddev=0.1)

# Create hidden layer
num_hiddennodes = 40

with tf.variable_scope('layer1'): 
    W_1 = tf.get_variable('W_1', [features,num_hiddennodes], 
                          initializer=weight_initializer)
    b_1 = tf.get_variable('b_1', [num_hiddennodes],
                          initializer=tf.constant_initializer(0.0))
    
    with tf.variable_scope('layer1_output'):
        l_1 = tf.matmul(X_ph, W_1) + b_1
        l_1 = tf.nn.relu(l_1)

# Create softmax layer
with tf.variable_scope('layer2'): 
    W_2 = tf.get_variable('W_2', [num_hiddennodes, num_classes], 
                          initializer=weight_initializer)
    b_2 = tf.get_variable('b_2', [num_classes],
                          initializer=tf.constant_initializer(0.0))
    
    with tf.variable_scope('layer2_output'):
         l_2 = tf.matmul(l_1, W_2) + b_2
              
Y = tf.nn.softmax(l_2)

# Print number of trainable parameters
print('Model consits of ', utils.num_params(), 'trainable parameters.')

Model consits of  13763 trainable parameters.


In [6]:
# Define prediction function
def pred(X_in, sess):
    feed_dict = {X_ph: X_in}
    fetches = [Y]
    res = sess.run(fetches, feed_dict)
    return res[0]

In [ ]:
## Implement training ops

# Define the cross entropy loss
with tf.variable_scope('loss'):
    
    # Compute loss   
    cross_entropy = -tf.reduce_sum(Y_ph * tf.log(Y), reduction_indices=[1]) 
    cross_entropy = tf.reduce_mean(cross_entropy)

    # L2 regularization
    reg_scale = 0.01
    regularize = tf.contrib.layers.l2_regularizer(reg_scale)
    params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    reg_term = sum([regularize(param) for param in params])
    cross_entropy += reg_term

# Define the training op
with tf.variable_scope('trainOP'):
    
    # Apply Adam optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    
    # Clip gradients
    gvs = optimizer.compute_gradients(cross_entropy)
    capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
    train_op = optimizer.apply_gradients(capped_gvs)

# Define the accuracy op
with tf.variable_scope('performance'):
       
    # Compute accuracy
    correct_prediction = tf.equal(tf.argmax(Y, axis=1), tf.argmax(Y_ph, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
## Start the session
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts))

if load_model:
    try:
        tf.train.Saver().restore(sess, "/save/model.ckpt")
        print("Using saved model")
    except:
        sess.run(tf.global_variables_initializer())
        print('Model not found, new parameters initialized')
else:
    sess.run(tf.global_variables_initializer())

In [ ]:
# Define parameters
max_epochs = 20000

train_cost, val_cost, train_acc, val_acc = [],[],[],[]

try:       
    for e in range(max_epochs):
            
        ## Fetch random batches
        batchsize_train = 300 # Number of amino acids
        batchsize_valid = 150
        
        # Get random indices 
        idx_train = np.random.choice(range(len(X_train_windows_masked)),batchsize_train,replace=False)
        idx_valid = np.random.choice(range(len(X_valid_windows_masked)),batchsize_valid,replace=False)

        # Get X batches
        X_train_batch = X_train_windows_masked[idx_train,:]       
        X_valid_batch = X_valid_windows_masked[idx_valid,:]
        
        # Get Y batches
        Y_train_batch = Y_train[idx_train,:]
        Y_valid_batch = Y_valid[idx_valid,:] 
                        
        # 1) Run the train op
        feed_dict_train = {X_ph: X_train_batch, Y_ph: Y_train_batch}
        fetches_train = [train_op, cross_entropy, accuracy]
        res = sess.run(fetches=fetches_train, feed_dict=feed_dict_train)
            
        # 2) Compute train_cost, val_cost, train_acc, val_acc
        train_cost += [res[1]]
        train_acc += [res[2]]
            
        # 3) Run validation
        feed_dict_valid = {X_ph: X_valid_batch, Y_ph: Y_valid_batch}
        fetches_valid = [cross_entropy, accuracy]
        res = sess.run(fetches=fetches_valid, feed_dict=feed_dict_valid)
            
        val_cost += [res[0]]
        val_acc += [res[1]]
            
        # Print training summaries
        if e % 1000 == 0:
            print("Epoch %i, Train Cost: %0.3f\tVal Cost: %0.3f\t Val acc: %0.3f" \
                %(e, train_cost[-1],val_cost[-1],val_acc[-1]))
            
            # Print intermediate predictions
            fetches_print = [Y]
            pred_intermediate = sess.run(fetches=fetches_print, feed_dict=feed_dict_train)
            print(np.shape(pred_intermediate))
            print(pred_intermediate)            

except KeyboardInterrupt:
    print('KeyboardInterrupt')

print('Done')

In [ ]:
# Define plot size
fig = plt.figure(figsize=(12,6))

# 1) Plot train and validation loss as a function of epochs
epoch = np.arange(len(train_cost))
fig.add_subplot(121)
plt.title('Loss')
plt.plot(epoch, train_cost,'r', label='Train Loss')
plt.plot(epoch, val_cost,'b', label='Val Loss')
plt.legend()
plt.xlabel('Epochs'), plt.ylabel('Loss')
plt.tight_layout()

# 2) Plot train and validation accuracy as a function of epochs
fig.add_subplot(122)
plt.title('Accuracy')
plt.plot(epoch, train_acc,'r', label='Train Accuracy')
plt.plot(epoch, val_acc,'b', label='Val Accuracy')
plt.legend(loc=4)
plt.xlabel('Epochs'), plt.ylabel('Accuracy')
plt.tight_layout()
plt.show()

In [ ]:
# Save model
save_path = tf.train.Saver().save(sess, "/tmp/model.ckpt")
print("Model saved in file: %s" % save_path)

In [ ]:
# Create confusion matrix from validation data results
feed_dict_valid = {X_ph: X_valid_windows_masked, Y_ph: Y_valid}
fetches = [Y]
preds = sess.run(fetches=fetches, feed_dict=feed_dict_valid)
preds = np.vstack(preds)
tmp = np.argmax(preds,axis=1).reshape(-1)
preds = np.eye(3)[tmp]

# Mask padding
boolmask = np.equal(np.sum(Y_valid,axis=1), 1).tolist()
Y_valid_masked = Y_valid[boolmask,:]
preds_masked = preds[boolmask,:]

# Compute metrics
preds_masked_dense = np.argmax(preds_masked, axis=1)
Y_valid_masked_dense = np.argmax(Y_valid_masked, axis=1)
confusionmat = confusion_matrix(Y_valid_masked_dense,preds_masked_dense)
print(confusionmat)
print("The total validation accuracy is: ",(confusionmat[0,0]+confusionmat[1,1]+confusionmat[2,2])/np.sum(confusionmat))
print("The random coil precision is: ",confusionmat[0,0]/np.sum(confusionmat[0,:]))
print("The random coil recall is: ",confusionmat[0,0]/np.sum(confusionmat[:,0]))
print("The beta sheet precision is: ",confusionmat[1,1]/np.sum(confusionmat[1,:]))
print("The beta sheet recall is: " ,confusionmat[1,1]/np.sum(confusionmat[:,1]))
print("The alpha helix precision is: ",confusionmat[2,2]/np.sum(confusionmat[2,:]))
print("The alpha helix recall is: ",confusionmat[2,2]/np.sum(confusionmat[:,2]))

In [ ]:
## Plot average magnitudes of weight groups
weights = np.zeros(windowsize*20)
weight_groups = np.empty(0)

# Fetch weight values
v = sess.run([v for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"layer1")])    

# Average and separate
for i in range(len(v[0])):
    weights[i] = np.mean(v[0][i])

for i in range(0,len(weights),20):
    weight_groups = np.append(weight_groups,np.sum(np.abs(weights[i:i+20])))

# Plot results
fig = plt.figure()
plt.plot(weight_groups, 'ro')
plt.xticks(range(17), [-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8])
plt.xlabel("Weight group position")
plt.ylabel("Average magnitude")
axes = plt.gca()
axes.set_ylim([min(weight_groups)-0.01,max(weight_groups)+0.01])
fig.show()